In [1]:
# air_quality_dashboard.ipynb

from environmental_insights import data as ei_data
import geopandas as gpd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

def launch_air_quality_widget():
    """
    Instantiate and display the pollutant→station→column interactive widgets,
    along with the map + time‐series outputs.
    """
    # 1) Load the UK outline once (for mapping)
    uk_outline = ei_data.get_uk_grids_outline()

    # 2) Define available pollutants
    available_pollutants = ["no2", "nox", "no", "o3", "pm10", "pm2.5", "so2", "co"]

    # 3) Dropdown for pollutant selection
    pollutant_dropdown = widgets.Dropdown(
        options=[("Select pollutant", None)] + [(pol.upper(), pol) for pol in available_pollutants],
        value=None,
        description="Pollutant:",
        style={"description_width": "initial"},
    )

    # 4) Dropdown for station selection (initially hidden)
    station_dropdown = widgets.Dropdown(
        options=[("Select station", None)],
        value=None,
        description="Station:",
        style={"description_width": "initial"},
    )
    station_dropdown.layout.display = "none"
    station_dropdown.disabled = True

    # 5) Dropdown for column selection (initially hidden)
    column_dropdown = widgets.Dropdown(
        options=[("Select column", None)],
        value=None,
        description="Column:",
        style={"description_width": "initial"},
    )
    column_dropdown.layout.display = "none"
    column_dropdown.disabled = True

    # 6) Spinner for loading indication
    spinner = widgets.HTML(
        value="""
        <style>
        @keyframes spin {
          0% { transform: rotate(0deg); }
          100% { transform: rotate(360deg); }
        }
        .loader {
          border: 4px solid #f3f3f3;
          border-top: 4px solid #444;
          border-radius: 50%;
          width: 20px;
          height: 20px;
          animation: spin 1s linear infinite;
          display: inline-block;
          vertical-align: middle;
        }
        .loading-text {
          display: inline-block;
          margin-left: 8px;
          vertical-align: middle;
          font-style: italic;
          color: #444;
        }
        </style>
        <div class="loader"></div>
        <div class="loading-text">Loading...</div>
        """
    )
    spinner.layout.display = "none"  # hidden initially

    # 7) Output areas for text, map, and time series
    output_area = widgets.Output()
    map_output = widgets.Output()
    ts_output = widgets.Output()

    # 8) Keep track of the last fetched station GeoDataFrame
    state = {"last_station_gdf": None}

    # 9) Callback: when pollutant changes, fetch stations
    def on_change_pollutant(change):
        if change["type"] == "change" and change["name"] == "value":
            selected_pollutant = change["new"]

            # Hide/reset station & column dropdowns, clear outputs
            station_dropdown.layout.display = "none"
            station_dropdown.disabled = True
            station_dropdown.options = [("Select station", None)]
            station_dropdown.value = None

            column_dropdown.layout.display = "none"
            column_dropdown.disabled = True
            column_dropdown.options = [("Select column", None)]
            column_dropdown.value = None

            with output_area:
                clear_output()
            with map_output:
                clear_output()
            with ts_output:
                clear_output()
            spinner.layout.display = "none"

            if selected_pollutant is None:
                return

            # Show station dropdown (greyed out while loading)
            station_dropdown.layout.display = ""
            station_dropdown.disabled = True

            # Show spinner
            spinner.layout.display = ""
            with output_area:
                clear_output()
                print(f"Fetching stations for {selected_pollutant.upper()}...")

            # Fetch station list (may take a moment)
            stations = ei_data.get_uk_monitoring_stations(selected_pollutant)

            # Hide spinner
            spinner.layout.display = "none"

            # Populate station dropdown
            placeholder = [("Select station", None)]
            station_options = placeholder + [(s, s) for s in stations]
            station_dropdown.options = station_options
            station_dropdown.value = None
            station_dropdown.disabled = False

            with output_area:
                clear_output()
                print(f"Fetched {len(stations)} station(s) measuring {selected_pollutant.upper()}.")
                print("Please select a station from the dropdown.")

    # 10) Callback: when station changes, fetch geometry & draw map
    def on_change_station(change):
        if change["type"] == "change" and change["name"] == "value":
            selected_station = change["new"]
            selected_pollutant = pollutant_dropdown.value

            # Hide/reset column dropdown, clear timeseries
            column_dropdown.layout.display = "none"
            column_dropdown.disabled = True
            column_dropdown.options = [("Select column", None)]
            column_dropdown.value = None

            with output_area:
                clear_output()
                if selected_station is None:
                    print(f"No station selected for {selected_pollutant.upper()}.")
                    return
                print(
                    f"You have selected:\n"
                    f"  • Pollutant: {selected_pollutant.upper()}\n"
                    f"  • Station:   {selected_station}"
                )

            # Show spinner
            spinner.layout.display = ""
            with map_output:
                clear_output()
            with ts_output:
                clear_output()

            # Fetch station geometry
            station_gdf = ei_data.get_uk_monitoring_station(
                pollutant=selected_pollutant, station=selected_station
            )
            state["last_station_gdf"] = station_gdf

            with map_output:
                clear_output()
                if station_gdf.empty:
                    spinner.layout.display = "none"
                    print("Station geometry not found.")
                else:
                    # Plot UK outline + station point
                    fig, ax = plt.subplots(figsize=(6, 6))
                    uk_outline.plot(ax=ax, facecolor="none", edgecolor="black", linewidth=0.8)
                    station_gdf.plot(
                        ax=ax, marker="x", color="red", markersize=100, label=selected_station
                    )
                    for _, row in station_gdf.iterrows():
                        pt = row.geometry
                        ax.text(pt.x + 2000, pt.y + 2000, selected_station, fontsize=10, zorder=6)
                    ax.set_title(f"{selected_station} Location", fontsize=12)
                    ax.axis("off")
                    plt.tight_layout()
                    plt.show()
                    spinner.layout.display = "none"

            # Show column dropdown if the station has data columns
            if not station_gdf.empty:
                possible_columns = [
                    col for col in station_gdf.columns
                    if col not in ["geometry", "Timestamp", "timestamp"]
                ]
                if possible_columns:
                    placeholder = [("Select column", None)]
                    column_options = placeholder + [(col, col) for col in possible_columns]
                    column_dropdown.options = column_options
                    column_dropdown.value = None
                    column_dropdown.layout.display = ""
                    column_dropdown.disabled = False

                    with output_area:
                        print("Please select a column to plot from the dropdown below.")

    # 11) Callback: when column changes, plot timeseries
    def on_change_column(change):
        if change["type"] == "change" and change["name"] == "value":
            selected_column = change["new"]
            selected_station = station_dropdown.value
            selected_pollutant = pollutant_dropdown.value

            with output_area:
                clear_output()
                if selected_column is None:
                    print("No column selected.")
                    return
                print(
                    f"You have selected:\n"
                    f"  • Pollutant: {selected_pollutant.upper()}\n"
                    f"  • Station:   {selected_station}\n"
                    f"  • Column:    {selected_column}"
                )

            spinner.layout.display = ""

            station_gdf = state["last_station_gdf"]
            if station_gdf is None or station_gdf.empty:
                with ts_output:
                    clear_output()
                    print("Station data not available for plotting.")
                spinner.layout.display = "none"
                return

            with ts_output:
                clear_output()
                if "Timestamp" in station_gdf.columns:
                    times = station_gdf["Timestamp"]
                elif "timestamp" in station_gdf.columns:
                    times = station_gdf["timestamp"]
                else:
                    station_gdf = station_gdf.set_index(station_gdf.index)
                    times = station_gdf.index

                measurements = station_gdf[selected_column]
                fig, ax_ts = plt.subplots(figsize=(8, 4))
                ax_ts.plot(times, measurements, linewidth=1.0)
                ax_ts.set_title(f"{selected_column}", fontsize=12)
                ax_ts.set_xlabel("Time", fontsize=10)
                ax_ts.set_ylabel(selected_column, fontsize=10)
                ax_ts.grid(alpha=0.3)
                plt.tight_layout()
                plt.show()

                spinner.layout.display = "none"

    # 12) Wire up callbacks
    pollutant_dropdown.observe(on_change_pollutant)
    station_dropdown.observe(on_change_station)
    column_dropdown.observe(on_change_column)

    # 13) Display all widgets in order
    display(
        pollutant_dropdown,
        station_dropdown,
        spinner,
        output_area,
        map_output,
        column_dropdown,
        ts_output
    )

    return {
        "pollutant": pollutant_dropdown,
        "station": station_dropdown,
        "column": column_dropdown,
        "spinner": spinner,
        "output": output_area,
        "map": map_output,
        "timeseries": ts_output,
    }

# Finally, call the function so that Voilà will render the UI:
widgets_dict = launch_air_quality_widget()


Dropdown(description='Pollutant:', options=(('Select pollutant', None), ('NO2', 'no2'), ('NOX', 'nox'), ('NO',…

Dropdown(description='Station:', disabled=True, layout=Layout(display='none'), options=(('Select station', Non…

HTML(value='\n        <style>\n        @keyframes spin {\n          0% { transform: rotate(0deg); }\n         …

Output()

Output()

Dropdown(description='Column:', disabled=True, layout=Layout(display='none'), options=(('Select column', None)…

Output()